In [1]:
import pandas as pd
import os

In [2]:
# connect to AWS RDS database

import psycopg2
connection = psycopg2.connect(
    host = 'covid-db.cz2dzm0eyvy5.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'covidwarriors',
    password = 'covidwarriors',
    database='project'
    )
cursor=connection.cursor()

In [3]:
# create the covid table to store csv

cursor.execute("""CREATE TABLE nc_covid(
case_month date,
res_state varchar,
state_fips_code int,
res_county varchar,
county_fips_code float,
age_group varchar,
sex varchar,
race varchar,
ethnicity varchar,
case_positive_specimen_interval float,
current_status varchar,
symptom_status varchar,
hosp_yn varchar,
death_yn varchar)""")


connection.commit()

In [4]:
# inspect the table to ensure set up 

sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,nc_covid,race,character varying,public
1,nc_covid,age_group,character varying,public
2,nc_covid,res_county,character varying,public
3,nc_covid,case_month,date,public
4,nc_covid,state_fips_code,integer,public
5,nc_covid,death_yn,character varying,public
6,nc_covid,ethnicity,character varying,public
7,nc_covid,sex,character varying,public
8,nc_covid,current_status,character varying,public
9,nc_covid,case_positive_specimen_interval,double precision,public


In [6]:
# add csv file to teh database

with open('Resources/nc_covid_data_updated.csv', 'r') as row:
    next(row)# Skip the header row.
    cursor.copy_from(row, 'nc_covid', sep=',')
    
    
connection.commit()

In [27]:
# make test queries 

sql = """
SELECT COUNT(hosp_yn)
FROM nc_covid
WHERE hosp_yn='Yes'
"""

pd.read_sql(sql, con=connection)

,count
0,303
